In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import json
from src.lang_agent import make_agent
from dotenv import load_dotenv
from typing import List

import json
import redis
import time
import os
from uuid import uuid4
from src import settings

from langchain.schema import (
    BaseMessage,
    HumanMessage,
    AIMessage,
    SystemMessage,
    messages_to_dict,
)

In [17]:
# Connect to Redis and assign to variable `db``
# Make use of settings.py module to get Redis settings like host, port, etc.
db = redis.Redis(settings.REDIS_IP, settings.REDIS_PORT, db=settings.REDIS_DB_ID)
db.ping()

True

In [18]:
def agent_predict(job_id:str, chat_history:List[BaseMessage])->List[BaseMessage]:
    """
    Receives the name of the user job_id and queues the job into Redis.
    Will loop until getting the answer from our retriver-generative service.

    Parameters
    ----------
    job_id: str
        id of the task
    messages : list[BaseMessage]
        chat_history

    Returns
    -------
    answer:  str
        Model generate an answer as a string 
    """
    
    # Assign an unique ID for this job and add it to the queue.
    # We need to assing this ID because we must be able to keep track
    # of this particular job across all the services
    job_id = str(uuid4())

    # Create a dict with the job data we will send through Redis having the   
    Out_dict = {
        "id": job_id,
        "chat_history":chat_history,
    }
    job_data = json.dumps(Out_dict)

    # Send the job to the model service using Redis 
    db.lpush(settings.REDIS_QUEUE, job_data)

    # Loop until we received the response from the retriver-generative model
    while True:
        # Attempt to get model predictions using job_id        
        output = db.get(job_id)

        # Check if the text was correctly processed by our model
        if output is not None:
            answer = json.loads(output)['answer']
            
            db.delete(job_id)
            break

        # Sleep some time waiting for model results
        time.sleep(settings.SERVER_SLEEP)

    return answer

In [20]:
# simule user interaction
def run_chatbox():
    chat_history = []
    job_id = str(uuid4())
    while True:
        query = input('User: ')
        if query == 'exit': 
            return output, chat_history
        chat_history.append(HumanMessage(content=query))

        # Run the agent with the query and chat history
        output = agent_predict(job_id, messages_to_dict(chat_history))
        print(f"Query:{query},\nAnswer: {output}")

        # updating chat history
        chat_history.append(AIMessage(content=output))
        

output, history = run_chatbox()

Query:What year was Atlassian founded?,
Answer: Atlassian was founded in 2002.
Query:Where is the headquarters?,
Answer: Atlassian's headquarters is located in Sydney, Australia. They also have offices around the world, including in the San Francisco Bay Area, California, Austin, Texas, the Netherlands, Japan, and other locations.
Query:What is the primary business?,
Answer: I apologize for the confusion earlier. Atlassian's primary business is developing and selling software for software developers, project managers, and content management. Their products include Jira, Confluence, Bitbucket, and Trello, among others.


In [21]:
history

[HumanMessage(content='What year was Atlassian founded?', additional_kwargs={}, example=False),
 AIMessage(content='Atlassian was founded in 2002.', additional_kwargs={}, example=False),
 HumanMessage(content='Where is the headquarters?', additional_kwargs={}, example=False),
 AIMessage(content="Atlassian's headquarters is located in Sydney, Australia. They also have offices around the world, including in the San Francisco Bay Area, California, Austin, Texas, the Netherlands, Japan, and other locations.", additional_kwargs={}, example=False),
 HumanMessage(content='What is the primary business?', additional_kwargs={}, example=False),
 AIMessage(content="I apologize for the confusion earlier. Atlassian's primary business is developing and selling software for software developers, project managers, and content management. Their products include Jira, Confluence, Bitbucket, and Trello, among others.", additional_kwargs={}, example=False)]